In [3]:

from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt
from sklearn.metrics import f1_score

from nltk import word_tokenize, sent_tokenize
import nltk
# nltk.download('punkt')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# from gensim.models.word2vec import Word2Vec
# from gensim.models import KeyedVectors

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, GlobalMaxPooling1D, Bidirectional, GRU, CuDNNGRU
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [0]:
datos = pd.read_csv('/content/gdrive/My Drive/train.csv')
X = datos['text']
y = datos['target']

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

In [55]:
vocab_size = 1000+1 # Adding 1 because of reserved 0 index
tokenizer = Tokenizer(num_words=vocab_size-1)
tokenizer.fit_on_texts(X)
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)

print(X_train[2])
print(X_val[3])

[296, 15, 255, 26, 4, 62, 7, 50, 402, 539, 4, 237, 181]
[14, 305, 14, 533, 20, 284, 4, 14, 136, 44, 537, 21, 207, 810, 7, 313, 7, 223, 165, 190, 5]


In [0]:
maxlen=200
X_train_padded = pad_sequences(X_train,padding='post',maxlen=maxlen)
X_val_padded   = pad_sequences(X_val, padding = 'post', maxlen = maxlen)

In [0]:
embedding_dim = 300

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(CuDNNGRU(128))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='relu'))
model.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])

In [58]:
# train
history = model.fit(X_train_padded, y_train,batch_size=30, epochs=5,validation_data=(X_val_padded, y_val)) 

Train on 6090 samples, validate on 1523 samples
Epoch 1/5
6090/6090 [==============================] - 5s 750us/step - loss: 6.9607 - acc: 0.5681 - val_loss: 6.7838 - val_acc: 0.5791
Epoch 2/5
6090/6090 [==============================] - 3s 434us/step - loss: 6.9607 - acc: 0.5681 - val_loss: 6.7838 - val_acc: 0.5791
Epoch 3/5
6090/6090 [==============================] - 3s 430us/step - loss: 6.9607 - acc: 0.5681 - val_loss: 6.7838 - val_acc: 0.5791
Epoch 4/5
6090/6090 [==============================] - 3s 421us/step - loss: 6.9607 - acc: 0.5681 - val_loss: 6.7838 - val_acc: 0.5791
Epoch 5/5
6090/6090 [==============================] - 3s 443us/step - loss: 6.9607 - acc: 0.5681 - val_loss: 6.7838 - val_acc: 0.5791


In [0]:
embedding_dim = 200
model_bigru = Sequential()
model_bigru.add(Embedding(vocab_size, embedding_dim, input_length=maxlen))
model_bigru.add(Bidirectional(CuDNNGRU(128, return_sequences = True)))
model_bigru.add(Bidirectional(CuDNNGRU(128)))
model_bigru.add(Dense(10, activation='relu'))
model_bigru.add(Dense(10, activation='relu'))
model_bigru.add(Dense(1, activation='sigmoid'))
model_bigru.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])

In [60]:
history = model_bigru.fit(X_train_padded, y_train ,batch_size=100, epochs=15, validation_data=(X_val_padded, y_val)) 

Train on 6090 samples, validate on 1523 samples
Epoch 1/15
6090/6090 [==============================] - 6s 1ms/step - loss: 0.5701 - acc: 0.6883 - val_loss: 0.4632 - val_acc: 0.7978
Epoch 2/15
6090/6090 [==============================] - 4s 639us/step - loss: 0.4192 - acc: 0.8159 - val_loss: 0.4520 - val_acc: 0.7971
Epoch 3/15
6090/6090 [==============================] - 4s 644us/step - loss: 0.3751 - acc: 0.8414 - val_loss: 0.4720 - val_acc: 0.7866
Epoch 4/15
6090/6090 [==============================] - 4s 643us/step - loss: 0.3508 - acc: 0.8580 - val_loss: 0.4891 - val_acc: 0.7886
Epoch 5/15
6090/6090 [==============================] - 4s 643us/step - loss: 0.3343 - acc: 0.8649 - val_loss: 0.5219 - val_acc: 0.7925
Epoch 6/15
6090/6090 [==============================] - 4s 642us/step - loss: 0.3058 - acc: 0.8862 - val_loss: 0.5368 - val_acc: 0.7787
Epoch 7/15
6090/6090 [==============================] - 4s 650us/step - loss: 0.2913 - acc: 0.8924 - val_loss: 0.5551 - val_acc: 0.7748
Ep